In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from pathlib import Path
from kedro.framework.context import load_context
import pandas as pd


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

current_dir = Path.cwd()  # this points to 'notebooks/' folder
proj_path = current_dir.parent  # point back to the root of the project
context = load_context(proj_path)
catalog = context.catalog

from matplotlib import pyplot as plt 
import seaborn as sns


In [ ]:
from phantasyfootballer.settings import *
from phantasyfootballer.common import Stats

In [ ]:
# Start by loading up the PPR data
df_all = catalog.load('scoring.ppr')
df_all.head()

But mostly what I care about is the points and the relative spot with regard to the position, so let's create 5 more views of the data, one with just the points, rankings and value fields.  And then a dataframe for each skill position

In [ ]:
df_value = df_all[[PLAYER_NAME, POSITION]+Stats.VALUE_STATS]
rb_value = df_value[df_value[POSITION] == RB].sort_values(Stats.FANTASY_POINTS, ascending=False)
qb_value = df_value[df_value[POSITION] == QB].sort_values(Stats.FANTASY_POINTS, ascending=False)
wr_value = df_value[df_value[POSITION] == WR].sort_values(Stats.FANTASY_POINTS, ascending=False)
te_value = df_value[df_value[POSITION] == WR].sort_values(Stats.FANTASY_POINTS, ascending=False)
flex_value = df_value.query('position in ["RB", "TE", "WR"]').sort_values(Stats.FANTASY_POINTS, ascending=False)

In [ ]:
flex_value.head()

In [ ]:
def get_top_players_by_stat(df_pos, stat, num_players=100):
    return df_pos.sort_values(stat,ascending=False).iloc[:num_players]
    
def plot_top_players_by_stat(df_pos, stat, num_players=100):
    '''
    Plot or display top players by stat
    '''
    players = get_top_players_by_stat(df_pos, stat, num_players=num_players)
    players.plot(kind='bar', x=PLAYER_NAME, y=stat);




In [ ]:
#plot_top_players_by_stat(flex_value, Stats.PCT_MEAN_OVR, num_players=50)
#get_top_players_by_stat(flex_value, Stats.PCT_MEAN_OVR, num_players=50)
get_top_players_by_stat(df_value, Stats.PCT_MEDIAN_OVR, num_players=50)
get_top_players_by_stat(df_value, Stats.PCT_MEAN_OVR, num_players=50)